<a href="https://colab.research.google.com/github/DenislavDyakov/SMuLe-Hatespeech-Detection/blob/main/Multilingual_Toxic_Comment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

In [2]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.7.58.210:8470
INFO:tensorflow:Initializing the TPU system: grpc://10.7.58.210:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.7.58.210:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
train = pd.read_csv('/content/gdrive/My Drive/jigsaw/jigsaw-toxic-comment-train.csv')
validation = pd.read_csv('/content/gdrive/My Drive/jigsaw/validation.csv')
test = pd.read_csv('/content/gdrive/My Drive/jigsaw/test.csv')

In [5]:
train.drop(['severe_toxic','obscene','threat','insult','identity_hate'],axis=1,inplace=True)

train = train.loc[:12000,:]
train.shape

In [6]:
!pip install transformers

     |████████████████████████████████| 1.3MB 3.4MB/s 
     |████████████████████████████████| 2.9MB 41.1MB/s 
     |████████████████████████████████| 890kB 44.4MB/s 
     |████████████████████████████████| 1.1MB 35.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=58dd5f748957ebeb4ae5e5389e4342ac871657c55e6848033a49821d82a1b7fe
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [7]:
# Loading Dependencies
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers

from tokenizers import BertWordPieceTokenizer

In [8]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=512):
    """
    Encoder for encoding the text into sequence of integers for BERT Input
    """
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding()
    all_ids = []
    
    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

In [9]:
#IMP DATA FOR CONFIG

AUTO = tf.data.experimental.AUTOTUNE


# Configuration
EPOCHS = 3
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 192

In [10]:
# First load the real tokenizer
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
# Save the loaded tokenizer locally
tokenizer.save_pretrained('.')
# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=False)
fast_tokenizer

Tokenizer(vocabulary_size=119547, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=False, wordpieces_prefix=##)

In [11]:
x_train = fast_encode(train.comment_text.astype(str), fast_tokenizer, maxlen=MAX_LEN)
x_valid = fast_encode(validation.comment_text.astype(str), fast_tokenizer, maxlen=MAX_LEN)
x_test = fast_encode(test.content.astype(str), fast_tokenizer, maxlen=MAX_LEN)

y_train = train.toxic.values
y_valid = validation.toxic.values

100%|██████████| 250/250 [00:15<00:00, 16.58it/s]


In [12]:
def build_model(transformer, max_len=512):
    """
    function for training the BERT model
    """
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [14]:
%%time
with strategy.scope():
    transformer_layer = (
        transformers.TFDistilBertModel
        .from_pretrained('distilbert-base-multilingual-cased')
    )
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()

Some layers from the model checkpoint at distilbert-base-multilingual-cased were not used when initializing TFDistilBertModel: ['vocab_transform', 'activation_13', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 192)]             0         
_________________________________________________________________
tf_distil_bert_model_1 (TFDi ((None, 192, 768),)       134734080 
_________________________________________________________________
tf_op_layer_strided_slice_1  [(None, 768)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 769       
Total params: 134,734,849
Trainable params: 134,734,849
Non-trainable params: 0
_________________________________________________________________
CPU times: user 7.39 s, sys: 17.8 s, total: 25.2 s
Wall time: 49.7 s


In [15]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

In [16]:
valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

In [44]:
 model.load_weights('/content/gdrive/My Drive/jigsaw/model_weights.h5')

In [17]:
n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS
)

Epoch 1/3
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


   2/1746 [..............................] - ETA: 4:06:17 - loss: 0.7036 - accuracy: 0.4922WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0059s vs `on_train_batch_end` time: 0.1272s). Check your callbacks.


1746/1746 [==============================] - ETA: 0s - loss: 0.1290 - accuracy: 0.9490WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0026s vs `on_test_batch_end` time: 0.0353s). Check your callbacks.


1746/1746 [==============================] - 263s 151ms/step - loss: 0.1290 - accuracy: 0.9490 - val_loss: 0.4734 - val_accuracy: 0.8479
Epoch 2/3
1746/1746 [==============================] - 236s 135ms/step - loss: 0.0930 - accuracy: 0.9620 - val_loss: 0.4384 - val_accuracy: 0.8504
Epoch 3/3
1746/1746 [==============================] - 237s 136ms/step - loss: 0.0803 - accuracy: 0.9665 - val_loss: 0.4771 - val_accuracy: 0.8505


In [18]:
n_steps = x_valid.shape[0] // BATCH_SIZE
train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    epochs=EPOCHS*2
)

Epoch 1/6
 2/62 [..............................] - ETA: 7s - loss: 0.3930 - accuracy: 0.8789WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0073s vs `on_train_batch_end` time: 0.1252s). Check your callbacks.


62/62 [==============================] - 8s 134ms/step - loss: 0.3139 - accuracy: 0.8655
Epoch 2/6
62/62 [==============================] - 21s 342ms/step - loss: 0.2286 - accuracy: 0.8998
Epoch 3/6
62/62 [==============================] - 8s 133ms/step - loss: 0.1766 - accuracy: 0.9217
Epoch 4/6
62/62 [==============================] - 8s 133ms/step - loss: 0.1219 - accuracy: 0.9477
Epoch 5/6
62/62 [==============================] - 8s 132ms/step - loss: 0.0907 - accuracy: 0.9632
Epoch 6/6
62/62 [==============================] - 8s 133ms/step - loss: 0.0838 - accuracy: 0.9657


In [20]:
test['toxic'] = model.predict(test_dataset, verbose=1)
test.to_csv('submission.csv', index=False)

  1/499 [..............................] - ETA: 0sWARNING:tensorflow:Callbacks method `on_predict_batch_end` is slow compared to the batch time (batch time: 0.0038s vs `on_predict_batch_end` time: 0.0332s). Check your callbacks.


499/499 [==============================] - 17s 35ms/step


In [23]:
testGrigor = pd.read_csv('/content/gdrive/My Drive/jigsaw/TestGrigor.csv',sep='\t')

In [32]:
testGrigor

,content,toxic
0,"Не го понасям този Гаун. Отишъл в Антверпен, а...",0.000250
1,САМО ДОКАТО ПРОЧЕТА ЗАГЛАВИЕТО И МИ СЕ ДОКЪКА,0.425476
2,Необяснимо голям брой уинъри от бекхенд. Да не...,0.064067
3,Браво гнидо чуждопоклонник.,0.386685
4,Явно Глигор има як сфинктер щом го е пречупил ...,0.076735
5,И ти посрещаш всяка вечер ГОСТ-и отзад...,0.000972
6,"Стига с тоя Антверп, и в статиите за Лудогорец...",0.000076
7,"Избивате комплекси. Полезно е, но моля не тук!!!",0.464919
8,Сервиза му е слабото място на Григор. 56% вкар...,0.001080
9,"Та като казваш, драга ми Виолета волейболистка...",0.123187


In [25]:
x_testGrigor = fast_encode(testGrigor.content.astype(str), fast_tokenizer, maxlen=MAX_LEN)

100%|██████████| 1/1 [00:00<00:00, 39.95it/s]


In [26]:
testGrigor_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_testGrigor)
    .batch(BATCH_SIZE)
)

In [45]:
testGrigor['toxic'] = model.predict(testGrigor_dataset, verbose=1)
testGrigor.to_csv('testGrigorResult.csv', index=False)

1/1 [==============================] - 0s 51ms/step


In [46]:
testGrigor

,content,toxic
0,"Не го понасям този Гаун. Отишъл в Антверпен, а...",0.000250
1,САМО ДОКАТО ПРОЧЕТА ЗАГЛАВИЕТО И МИ СЕ ДОКЪКА,0.425476
2,Необяснимо голям брой уинъри от бекхенд. Да не...,0.064067
3,Браво гнидо чуждопоклонник.,0.386685
4,Явно Глигор има як сфинктер щом го е пречупил ...,0.076735
5,И ти посрещаш всяка вечер ГОСТ-и отзад...,0.000972
6,"Стига с тоя Антверп, и в статиите за Лудогорец...",0.000076
7,"Избивате комплекси. Полезно е, но моля не тук!!!",0.464919
8,Сервиза му е слабото място на Григор. 56% вкар...,0.001080
9,"Та като казваш, драга ми Виолета волейболистка...",0.123187


In [40]:
 #model.save_weights('model_weights.h5')